In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt

# 1. Melhorar a engenharia de features
def criar_features_avancadas(X):
    """
    Criar features avançadas para melhorar o modelo de radiação solar
    """
    X_novo = X.copy()
    
    # Features existentes
    X_novo['dist_equador'] = np.abs(X_novo['latitude'])
    X_novo['lat_mes_interact'] = X_novo['latitude'] * np.cos(2 * np.pi * X_novo['mes'] / 12)
    X_novo['declinacao_solar'] = 23.45 * np.sin(2 * np.pi * (X_novo['dia_ano'] - 81) / 365)
    
    # Novas features avançadas
    
    # Ângulo zenital solar (melhor estimativa da elevação solar)
    lat_rad = np.radians(X_novo['latitude'])
    dec_rad = np.radians(X_novo['declinacao_solar'])
    hora_rad = np.radians((X_novo['hora'] - 12) * 15)  # Convertendo hora para ângulo horário
    
    X_novo['cos_zenite'] = (np.sin(lat_rad) * np.sin(dec_rad) + 
                          np.cos(lat_rad) * np.cos(dec_rad) * np.cos(hora_rad))
    
    # Massa de ar óptica (quantidade de atmosfera que a luz atravessa)
    # Fórmula simplificada de Kasten and Young
    X_novo['massa_ar'] = 1 / (X_novo['cos_zenite'] + 0.50572 * (96.07995 - np.degrees(np.arccos(X_novo['cos_zenite'])))**-1.6364)
    X_novo['massa_ar'] = X_novo['massa_ar'].clip(1, 38)  # Limitando valores extremos
    
    # Impacto da altitude na radiação
    X_novo['fator_altitude'] = np.exp(X_novo['altitude'] / 8000)  # Aproximação do efeito da pressão atmosférica
    
    # Interações entre temperatura e umidade
    X_novo['temp_umid_interact'] = X_novo['temperatura_do_ar___bulbo_seco_horaria_degc'] * X_novo['umidade_relativa_do_ar_horaria_percent'] / 100
    
    # Fator de claridade do céu (estimativa baseada na umidade)
    X_novo['claridade_ceu'] = 1 - (X_novo['umidade_relativa_do_ar_horaria_percent'] / 100) * 0.75
    
    # Radiação potencial no topo da atmosfera
    const_solar = 1367  # W/m²
    distancia_terra_sol = 1 + 0.033 * np.cos(2 * np.pi * X_novo['dia_ano'] / 365)
    X_novo['radiacao_toa'] = const_solar * distancia_terra_sol * X_novo['cos_zenite']
    X_novo['radiacao_toa'] = X_novo['radiacao_toa'].clip(0, None)  # Eliminando valores negativos
    
    # Converter para kJ/m² por hora
    X_novo['radiacao_toa_kj'] = X_novo['radiacao_toa'] * 3.6
    
    return X_novo

# 2. Processamento de dados mais robusto
def processar_dados(X, y, treino=True, scaler=None, power_transformer=None):
    """
    Processamento mais robusto dos dados, incluindo normalização e transformação
    """
    # Normalização de todas as features numéricas
    features_numericas = X.select_dtypes(include=['float64', 'int64']).columns.tolist()
    
    if treino:
        # Inicializar transformadores
        scaler = StandardScaler()
        power_transformer = PowerTransformer(method='yeo-johnson')
        
        # Ajustar e transformar
        X[features_numericas] = scaler.fit_transform(X[features_numericas])
        
        # Transformar variável alvo para distribuição mais normal
        y_transformed = power_transformer.fit_transform(y.values.reshape(-1, 1)).flatten()
        
        return X, y_transformed, scaler, power_transformer
    else:
        # Usar transformadores pré-ajustados
        X[features_numericas] = scaler.transform(X[features_numericas])
        return X

# 3. Otimização de hiperparâmetros via GridSearchCV (exemplo simplificado)
def otimizar_modelo():
    """
    Otimização de hiperparâmetros para o modelo XGBoost
    """
    param_grid = {
        'n_estimators': [300, 500, 700],
        'max_depth': [6, 8, 10],
        'learning_rate': [0.01, 0.05, 0.1],
        'subsample': [0.7, 0.8, 0.9],
        'colsample_bytree': [0.7, 0.8, 0.9],
        'gamma': [0, 0.1, 0.2]
    }
    
    modelo_base = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
    
    grid_search = GridSearchCV(
        estimator=modelo_base,
        param_grid=param_grid,
        cv=5,
        scoring='neg_mean_squared_error',
        verbose=1,
        n_jobs=-1
    )
    
    return grid_search

# 4. Função de previsão melhorada
def previsao_radiacao_melhorada(modelo, latitude, longitude, altitude=500, ano=2024, 
                               scaler=None, power_transformer=None):
    """
    Versão melhorada da função de previsão de radiação solar
    """
    # Criar datas para o ano inteiro
    data_inicio = pd.Timestamp(f"{ano}-01-01")
    data_fim = pd.Timestamp(f"{ano}-12-31")
    datas = pd.date_range(data_inicio, data_fim, freq='D')
    
    # Dataframe para armazenar resultados
    resultados = []
    
    # Para cada dia, prever radiação ao meio-dia
    for data in datas:
        # Extrair componentes temporais
        mes = data.month
        dia_ano = data.dayofyear
        
        # Criar previsões para várias horas do dia para capturar o ciclo diário
        for hora in range(6, 19):  # Das 6h às 18h (período de luz)
            # Criar features cíclicas e outras
            dados = {
                'latitude': [latitude],
                'longitude': [longitude],
                'altitude': [altitude],
                'mes': [mes],
                'dia_ano': [dia_ano],
                'hora': [hora],
                'mes_sen': [np.sin(2 * np.pi * mes / 12)],
                'mes_cos': [np.cos(2 * np.pi * mes / 12)],
                'hora_sen': [np.sin(2 * np.pi * hora / 24)],
                'hora_cos': [np.cos(2 * np.pi * hora / 24)],
                'dia_ano_sen': [np.sin(2 * np.pi * dia_ano / 365)],
                'dia_ano_cos': [np.cos(2 * np.pi * dia_ano / 365)]
            }
            
            # Adicionar valores meteorológicos típicos para a data e hora
            # Aqui poderia ser aprimorado com dados meteorológicos mais precisos
            dados['temperatura_do_ar___bulbo_seco_horaria_degc'] = [get_temp_estimada(mes, hora, latitude)]
            dados['precipitacao_total_horario_mm'] = [get_precip_estimada(mes, latitude)]
            dados['umidade_relativa_do_ar_horaria_percent'] = [get_umid_estimada(mes, hora, latitude)]
            dados['vento_velocidade_horaria_m_s'] = [get_vento_estimado(mes)]
            
            # Criar DataFrame
            df_previsao = pd.DataFrame(dados)
            
            # Aplicar a engenharia de features avançada
            df_com_features = criar_features_avancadas(df_previsao)
            
            # Aplicar o processamento de dados
            df_processado = processar_dados(df_com_features, None, treino=False, 
                                           scaler=scaler, power_transformer=None)
            
            # Obter todas as colunas necessárias para o modelo
            colunas_modelo = modelo.feature_names_in_
            for col in colunas_modelo:
                if col not in df_processado.columns:
                    df_processado[col] = 0  # Valor padrão para colunas ausentes
            
            # Fazer previsão
            previsao_transformada = modelo.predict(df_processado[colunas_modelo])
            
            # Reverter a transformação se aplicada
            if power_transformer is not None:
                radiacao = power_transformer.inverse_transform(previsao_transformada.reshape(-1, 1)).flatten()[0]
            else:
                radiacao = previsao_transformada[0]
            
            # Adicionar a previsão ao resultado
            resultados.append({
                'data': data,
                'hora': hora,
                'radiacao_prevista': radiacao,
                'mes': mes,
                'dia_ano': dia_ano
            })
    
    # Criar DataFrame com resultados
    df_resultados = pd.DataFrame(resultados)
    
    # Calcular médias diárias
    df_diario = df_resultados.groupby('data').agg({
        'radiacao_prevista': 'mean',
        'mes': 'first',
        'dia_ano': 'first'
    }).reset_index()
    
    return df_diario

# Funções auxiliares para estimar condições meteorológicas
def get_temp_estimada(mes, hora, latitude):
    """Estima temperatura com base no mês, hora e latitude"""
    # Ajuste de base pela latitude (mais frio nos extremos)
    base_temp = 25 - 0.5 * abs(latitude)
    
    # Ajuste sazonal (hemisfério sul: inverno em jun/jul, verão em dez/jan)
    ajuste_sazonal = 8 * np.cos(2 * np.pi * (mes - 1) / 12 + (np.pi if latitude < 0 else 0))
    
    # Ajuste diurno (mais quente às 14h, mais frio às 5h)
    ajuste_diurno = 5 * np.sin(2 * np.pi * (hora - 5) / 24)
    
    return base_temp + ajuste_sazonal + ajuste_diurno

def get_precip_estimada(mes, latitude):
    """Estima precipitação com base no mês e latitude"""
    if latitude < 0:  # Hemisfério Sul
        if 11 <= mes <= 12 or 1 <= mes <= 3:  # Verão
            return np.random.gamma(2, 2)  # Mais chuvas
        else:
            return np.random.gamma(1, 1)  # Menos chuvas
    else:  # Hemisfério Norte
        if 5 <= mes <= 9:  # Verão
            return np.random.gamma(2, 2)
        else:
            return np.random.gamma(1, 1)

def get_umid_estimada(mes, hora, latitude):
    """Estima umidade com base no mês, hora e latitude"""
    # Base mais alta para regiões próximas ao equador
    base_umid = 70 - 0.5 * abs(latitude)
    
    # Ajuste sazonal
    ajuste_sazonal = 10 * np.cos(2 * np.pi * (mes - 1) / 12 + (np.pi if latitude < 0 else 0))
    
    # Ajuste diurno (mais úmido de madrugada, mais seco à tarde)
    ajuste_diurno = -15 * np.sin(2 * np.pi * (hora - 5) / 24)
    
    umidade = base_umid + ajuste_sazonal + ajuste_diurno
    return min(max(umidade, 30), 100)  # Limitar entre 30% e 100%

def get_vento_estimado(mes):
    """Estima velocidade do vento com base no mês"""
    # Ventos ligeiramente mais fortes nos meses de transição
    if mes in [3, 4, 9, 10]:
        return np.random.normal(4, 1)
    else:
        return np.random.normal(3, 1)

# 5. Exemplo de uso melhorado (pseudocódigo)
"""
# Criar e processar features
X_com_features = criar_features_avancadas(X)
X_treino, y_treino, scaler, power_transformer = processar_dados(X_com_features, y, treino=True)

# Otimizar modelo
otimizador = otimizar_modelo()
otimizador.fit(X_treino, y_treino)
modelo_otimizado = otimizador.best_estimator_

# Fazer previsões
df_previsao = previsao_radiacao_melhorada(
    modelo_otimizado, 
    latitude=-2.53, 
    longitude=-44.30, 
    altitude=50, 
    scaler=scaler,
    power_transformer=power_transformer
)

# Avaliar resultados
plot_radiacao_anual(df_previsao)
"""

# 6. Fator de correção para calibração
def aplicar_calibracao(df_previsoes, fator_multiplicativo=2.3, offset=0):
    """
    Aplica um fator de calibração às previsões para ajustá-las aos valores reais
    """
    df_calibrado = df_previsoes.copy()
    df_calibrado['radiacao_prevista_calibrada'] = df_calibrado['radiacao_prevista'] * fator_multiplicativo + offset
    return df_calibrado

# Exemplo de uso da calibração
"""
df_previsao_calibrado = aplicar_calibracao(df_previsao, fator_multiplicativo=2.5)
"""